# Model Selection Practical Notebook

In this notebook you will demonstrate what you have learnt in the lesson and tackle the following challenges:

1. Write your own "Randomised Selection" code to select the optimal configuration for your linear model.
2. Modify the `SimpleNeuralNetwork()` code in the lesson notebook such that it runs a classificaiton model.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, load_diabetes, load_breast_cancer
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [2]:
# Preparing the data

diabetes = load_diabetes()
X = diabetes['data']
y = diabetes['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Q1.** Implement your own Randomised Selection function.

Recall that randomised selection is a method to determine the optimal configuration for a linear regression model.

_Hint: you would need a sampling function that gives you potential feature candidates, consider using `np.random.choice`. Remember also that AIC and BIC should only be calculated on the training sample, where you may also wish to calculate MSE on the test sample_

In [3]:
# AIC and BIC function

def AIC(X, y, lm):
    """ Compute the AIC score of a linear model"""
    N = X.shape[0]
    y_pred = lm.predict(X)
    ll = -(N/2)*np.log(mean_squared_error(y_pred, y))
    k = len(lm.coef_)
    
    return 2*k - 2*ll

def BIC(X, y, lm):
    """ Compute the BIC score of a linear model"""
    N = X.shape[0]
    y_pred = lm.predict(X)
    ll = -(N/2)*np.log(mean_squared_error(y_pred, y))
    k = len(lm.coef_)
    
    return k*np.log(N) - 2*ll

In [4]:
# Your code here ...

def sample_candidates(num_samples, p):
    """
    args:
        num_samples: number of samples to sample, if None, returns all possible combinations.
        p: number of features
    """
    
    all_samples = []
    
    for i in range(1,2**p):
        all_samples.append(np.array(list(np.binary_repr(i, width=p)), dtype=int))
    
    if num_samples: 
        indx = list(range(len(all_samples)))
        chosen_indx = np.random.choice(indx, size = num_samples)
        
        return np.array([all_samples[i] for i in chosen_indx]).astype(bool)

    else:
        return np.array(all_samples).astype(bool)

    

def return_metric(X_train, y_train, X_test, y_test, candidate, metric="aic"):
    """
    args:
        X_train, y_train, X_test, y_test: The standard train-validation test split of your training data.
        candidate: Candidate set of features for testing - can take either an array of unique column indexes or boolean array as to whether corresponding column should be included
        metric: the metric used to access the goodness of fit. "aic", "bic" or "mse". Note that AIC and BIC are calculated on the training data, whereas MSE is calculated on the test data.
    
    """
    
    X_train_sub = X_train[:, candidate]
    X_test_sub = X_test[:, candidate]
    lm = LinearRegression()
    lm.fit(X_train_sub, y_train)
    
    if metric == "aic":
        return AIC(X_train_sub, y_train, lm)
    elif metric == "bic":
        return BIC(X_train_sub, y_train, lm)
    elif metric == "mse":
        return mean_squared_error(y_test, lm.predict(X_test_sub))


def random_selection(X_train, y_train, X_test, y_test, metric, num_samples=None):
    """
    args:
        X_train, y_train, X_test, y_test: The standard train-validation test split of your training data.
        num_samples: number of samples for the `sample_candidate` function
        metric: the metric used to access the goodness of fit. "aic" or "bic"
    
    """
    samples = sample_candidates(num_samples, X_train.shape[1])
    metric_ls = []
    for candidate in samples:
        metric_ls.append(return_metric(X_train, y_train, X_test, y_test, candidate, metric=metric))
    
    return samples, metric_ls


# Demo
np.random.seed(42)
for metric in ['aic', 'bic', 'mse']:
    samples_, metric_ls_ = random_selection(X_train, y_train, X_test, y_test, metric, 30)
    best_sample = samples_[np.argmin(metric_ls_)]
    print(f"The optimal feature configuration according to the {metric.upper()} is: {best_sample.astype(int)}")




The optimal feature configuration according to the AIC is: [0 1 1 1 0 0 1 0 1 1]
The optimal feature configuration according to the BIC is: [0 1 1 1 0 1 1 0 1 1]
The optimal feature configuration according to the MSE is: [0 0 1 0 1 0 0 1 1 1]


## Detailed Annotation of Q1 Solution

### Overview
This solution implements a **randomized feature selection algorithm** that searches for the optimal subset of features for linear regression by testing random combinations and evaluating them using AIC, BIC, or MSE.

---

### Function 1: `sample_candidates(num_samples, p)`

**Purpose**: Generate feature subset candidates to test.

**Algorithm**:
1. **Binary enumeration trick**: Represents each feature combination as a binary number
   - For `p=3` features: `101` binary = use features 0 and 2, skip feature 1
   - Total combinations: `2^p - 1` (excluding the empty set)

2. **Generate all possible combinations**:
   ```python
   for i in range(1, 2**p):  # Start at 1 to avoid empty feature set
       np.binary_repr(i, width=p)  # Convert i to binary string with p digits
   ```
   - Example: `binary_repr(5, width=4)` → `"0101"` → `[0,1,0,1]` → `[False, True, False, True]`

3. **Random sampling**:
   - If `num_samples` specified: randomly select that many combinations
   - If `None`: return all combinations (exhaustive search)

**Returns**: Boolean array of shape `(num_samples, p)` where `True` = include feature

---

### Function 2: `return_metric(X_train, y_train, X_test, y_test, candidate, metric)`

**Purpose**: Evaluate a single feature subset using the specified metric.

**Steps**:
1. **Feature selection**: Use boolean indexing to select only the features indicated by `candidate`
   ```python
   X_train_sub = X_train[:, candidate]  # Keep only True columns
   ```

2. **Train model**: Fit `LinearRegression` on the reduced dataset

3. **Compute metric**:
   - **AIC** (Akaike Information Criterion): `2k - 2LL`
     - Balances model fit (log-likelihood) vs complexity (k parameters)
     - Computed on **training data**
     - Lower is better
   
   - **BIC** (Bayesian Information Criterion): `k·log(N) - 2LL`
     - Similar to AIC but penalizes complexity more heavily
     - Computed on **training data**
     - Lower is better
   
   - **MSE** (Mean Squared Error): Average squared prediction errors
     - Measures generalization performance
     - Computed on **test data**
     - Lower is better

**Key insight**: AIC/BIC assess model quality during fitting; MSE assesses prediction accuracy on unseen data.

---

### Function 3: `random_selection(X_train, y_train, X_test, y_test, metric, num_samples)`

**Purpose**: Main orchestrator that performs the randomized search.

**Algorithm**:
1. Generate `num_samples` random feature subsets using `sample_candidates()`
2. Evaluate each subset by calling `return_metric()`
3. Store all metric scores in a list
4. Return both the candidates and their scores

**Returns**: `(samples, metric_ls)` tuple
- `samples`: Array of boolean feature combinations tested
- `metric_ls`: List of corresponding metric scores

**Usage**: Find best configuration with `samples[np.argmin(metric_ls)]`

---

### Demo Section

**What it does**:
```python
np.random.seed(42)  # Ensures reproducibility
```
- Tests **30 random** feature combinations (out of 1024 possible for 10 features)
- Compares using three different metrics: AIC, BIC, and MSE
- Finds the best configuration for each metric
- Prints binary array showing which features to include (1 = include, 0 = exclude)

**Example output interpretation**:
```
[1 0 1 1 0 1 0 1 1 0]
```
Means: Use features 0, 2, 3, 5, 7, 8 (where value is 1)

---

### Efficiency Analysis

| Scenario | Features | Total Combinations | Randomized (30 samples) | Time Savings |
|----------|----------|-------------------|------------------------|--------------|
| Diabetes | 10 | 1,023 | 30 | ~97% |
| Medium | 15 | 32,767 | 30 | ~99.9% |
| Large | 20 | 1,048,575 | 30 | ~99.997% |

**Conclusion**: Randomized selection makes feature selection tractable for high-dimensional data while still finding near-optimal solutions.

**Q2.** Modify the regression code in `SimpleNeuralNetwork` into a classification model.

_Hint: Modify the `model` object defined in the `.fit()` method and also change the loss function from mean squared loss to binary cross entropy loss._

In [ ]:
# Prepare binary classfication data

cancer = load_breast_cancer()
X, y = cancer['data'], cancer['target'].reshape(-1, 1)

In [ ]:
import torch

# Regression code to modify
class SimpleNeuralNetwork():

    def __init__(self, h1, epoch, verbose=False):
        """
        args:
            h1: number of nodes in hidden layer 1
            epoch: number of gradient updates
        """
        self.epoch = epoch
        self.h1 = h1
        self.verbose = verbose

    def fit(self, X, y):
        """
        args:
            X, y: predictor and target
        """
        n, p = X.shape
        inputs = torch.from_numpy(X).float()
        targets = torch.from_numpy(y).float()

        # Create the neural network model
        model = torch.nn.Sequential(torch.nn.Linear(p, self.h1),
                                    torch.nn.ReLU(),
                                    torch.nn.Linear(self.h1, 1))
        loss_fn = torch.nn.MSELoss()
        opt = torch.optim.SGD(model.parameters(), lr=1e-3)

        for rd in range(self.epoch):
            pred = model(inputs)
            loss = loss_fn(pred, targets)

            if rd%300 == 0:
                print("loss: %.2f" %loss)

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.model = model

    def predict(self, x_test):
        inputs = torch.from_numpy(x_test).float()
        return self.model(inputs).data.numpy()


